In [ ]:
import pyabf
import matplotlib.pyplot as plt
import numpy as np

abf = pyabf.ABF("2022_05_18_0001.abf")

fig, axs = plt.subplots(8, 2, figsize=(20, 20)) # Create a figure with 8 rows and 2 columns of subplots

for i, ax in enumerate(axs.flatten()): # Loop over each channel and corresponding subplot axis
    abf.setSweep(sweepNumber=0, channel=i)
    data = abf.sweepY  # Get the data for the current channel
    time = abf.sweepX  # Get the sweepX data (time) for the current channel

    ax.plot(time, data, color='C0')
    ax.set_title(f'Raw Data - Channel {i + 1}')
    ax.set_xlabel(abf.sweepLabelX)
    ax.set_ylabel(abf.sweepLabelY)

plt.tight_layout()
plt.savefig('rawdata.png', dpi=300) # Save the figure as a PNG file with a resolution of 300 DPI
plt.close(fig)  # Close the figure to free up memory

In [ ]:
import torch
import torchaudio
import matplotlib.pyplot as plt
import pyabf

device = torch.device('cuda:1')  # Use my second GPU

def compute_spectrogram(data, fs):
    spectrogram = torchaudio.transforms.Spectrogram(n_fft=int(fs*0.4), # Create a Spectrogram object with specified parameters
    win_length=int(fs*0.4), hop_length=int(fs*0.2), power=2)
    Sxx = spectrogram(torch.tensor(data)) 
    Sxx = 10.0 * torch.log10(Sxx + 1e-9)
    Sxx = Sxx.to('cpu') # Move the spectrogram to the CPU
    return Sxx

fabf = pyabf.ABF("2022_05_18_0001.abf")
fig, axes = plt.subplots(8, 2,  figsize=(20, 20), sharex=True)     # Create a figure with 8 rows and 2 columns of subplots

for i, ax in enumerate(axes.flatten()):
    abf.setSweep(sweepNumber=0, channel=i)
    data = abf.sweepY  # Get the data for the current channel
    time = abf.sweepX  # Get the sweepX data (time) for the current channel
    fs = 1/(time[1] - time[0]) # Sampling frequency

    Sxx = compute_spectrogram(data, fs)
    cax = ax.imshow(Sxx, aspect='auto', origin='lower', cmap='inferno')  # Display the spectrogram as an image
    ax.set_title(f'Spectrogram - Channel {i + 1}')
    ax.set_ylabel('Frequency (Hz)')
    fig.colorbar(cax, ax=ax, label='Frequency (Hz)')         # Add a colorbar to the subplot

for ax in axes[-1, :]:
    ax.set_xlabel('Time (s)')

plt.tight_layout()
plt.savefig('spectrograms.png', dpi=300)
plt.close(fig)  # Close the figure to free up memory

In [ ]:
import numpy as np
import scipy.signal
import pyabf
import matplotlib.pyplot as plt

# Honestly I couldnt think of a better way to do this so I just used O(n^2)

# Function to apply a bandpass filter (Used to remove noise from the signal)
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs 

    # Normalize the cutoff frequencies by the Nyquist frequency
    low = lowcut / nyquist 
    high = highcut / nyquist

    b, a = scipy.signal.butter(order, [low, high], btype='band') # Compute the coefficients for a Butterworth filter

    # Apply the filter to the data using scipy.signal.lfilter
    filtered_data = scipy.signal.lfilter(b, a, data)

    return filtered_data

#Frequency bands of interest (I figured that delta and theta bands are not necessary since those are sleep)
bands = {'Alpha': (8, 12), 'Beta': (12, 30), 'Gamma': (30, 100)}

abf = pyabf.ABF("2022_05_18_0001.abf")

for band_name, (lowcut, highcut) in bands.items():     # Loop over each band
    fig, axes = plt.subplots(8, 2,  figsize=(20, 20), sharex=True)     # Create a figure for the current band

    for i, ax in enumerate(axes.flatten()):
        abf.setSweep(sweepNumber=0, channel=i)
        data = abf.sweepY  # Get the data for the current channel
        time = abf.sweepX  # Get the sweepX data (time) for the current channel
        fs = 1/(time[1] - time[0]) # Sampling frequency

        filtered_data = bandpass_filter(data, lowcut, highcut, fs)  # Filter the data for the current band
        ax.plot(time, filtered_data, label=f'{band_name} Band ({lowcut}-{highcut} Hz)')
        ax.set_title(f'Filtered Signals - Channel {i + 1}')
        ax.set_ylabel('Amplitude')
        ax.legend()

    for ax in axes[-1, :]:
        ax.set_xlabel('Time (s)')

    plt.tight_layout()
    plt.savefig(f'filtered_signals_{band_name}.png', dpi=300)
    plt.close(fig)  # Close the figure to free up memory